In [6]:
import pandas as pd
import time
from datetime import datetime

In [7]:
# Base url to append dates to
base_url = 'http://schiphol.dutchplanespotters.nl/?date='
# Create list of dates for 2017 and 2018 (730 days)
date_range = pd.date_range('2017-01-01 00:00:00', '2018-12-31 00:00:00', freq='D').strftime('%Y-%m-%d')
# Create list of urls for all days, by appending date to base url
urls = [base_url + date for date in date_range]


In [8]:
# Crawler: Pulls tables from the url list with one second delay after each request
# Saves to dataframe df
df = pd.DataFrame()
for i, url in enumerate(urls[:5]):
    # Pull info from url
    day_flights = pd.read_html(url)[0]
    
    # Get number of rows and current date
    num_rows = len(day_flights)
    curr_date = date_range[i]
    
    date_column = pd.Series([curr_date] * num_rows)
    day_flights['date'] = date_column
    
    df = df.append(day_flights)
    time.sleep(1)
    print('Request number: ', i)

In [19]:
# Writes crawled information to a csv file
open('schiphol-pulled-data.csv', 'w').write(df.to_csv())

In [40]:
df = pd.read_csv('schiphol-flights-2017-2018.csv')
df = df.drop(df.columns[0], axis=1)
df = df.drop(index=0)

In [41]:
# Change column names for semantics
column_names = ['Cargo', 'Aircraft', 'Airline', 'Registration', 'Remark', 'Gate', 'Arr_FlightNr', 'Origin', 'STA', 'ETA', 'Status', 'Dep_FlightNr', 'Destination', 'STD', 'ETD', 'Status', 'Date']
df.columns = column_names

In [42]:
# This function appends the date (%Y-%m-%d) to a specified column where records
# are missing this date
def add_date_to_time(col_name):
    df_copy = df.copy()
    mask = df_copy[col_name].str.contains('\d{2}:\d{2}\Z').fillna(False)

    df_copy['Date'] = pd.to_datetime(df_copy['Date'], format='%Y-%m-%d')
    date_string = df_copy['Date'].copy()
    date_string = date_string.loc[mask]
    date_string = pd.DatetimeIndex(date_string)
    date_string = date_string.strftime('(%d-%m-%y)')
    date_string

    df_copy.loc[mask, col_name] = df_copy.loc[mask, col_name].str.cat(date_string, sep =' ')
    return df_copy

# Run this function on our 4 columns
df['STA'] = add_date_to_time('STA')['STA']
df['ETA'] = add_date_to_time('ETA')['ETA']
df['STD'] = add_date_to_time('STD')['STD']
df['ETD'] = add_date_to_time('ETD')['ETD']

/home/kjw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: A future version of pandas will perform index alignment when `others` is a Series/Index/DataFrame (or a list-like containing one). To disable alignment (the behavior before v.0.23) and silence this warning, use `.values` on any Series/Index/DataFrame in `others`. To enable alignment and silence this warning, pass `join='left'|'outer'|'inner'|'right'`. The future default will be `join='left'`.
  


In [64]:
# Airline name translation
airline_url = 'http://www.sterlingtravel.co.uk/airline_list.html'
airline_table = pd.read_html(airline_url)[3]

In [65]:
airline_code_mapping = airline_table[['Airline name', '2-letter code']]

In [66]:
# We need to either map each airline name in the original df to the 2 letter code, or 2 letter code to airline name
# Former is probably the better option.
# NOTE: 2-letter codes are most likely identical between df and the mapping, but the names are not.
airline_code_mapping

,Airline name,2-letter code
0,Adria Airways,JP
1,Aegean Airlines,A3
2,Aer Arann,RE
3,Aer Lingus,EI
4,Aeroflot Russian Airlines,SU
5,Aerolineas Argentinas,AR
6,Aeromexico,AM
7,Air Algerie,AH
8,Air Astana,KC
9,Air Canada,AC
